In [35]:
# судя по всему, для парсинга расписания нам понадобиться доставать инфу через selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options

import time

# создаем драйвер
# следующие две строчки нужны для того, чтобы браузер не открывался
chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# url сайта рггу
url = 'https://www.rsuh.ru/raspis/'
driver.get(url)
time.sleep(2)

# просим пользователя ввести интересующие данные (в боте нужно будет по-другому это делать)
edu_form = input('Введите форму обучения: ')
course_num = input('Введите курс обучения: ')
period = input('Введите срок: ')
speciality = input('Введите название вашей специальности: ')

# в общем тут мы просто выставляем значения у выпадающих списков
for i in zip(['formob', 'kyrs', 'srok'], [edu_form, course_num, period]):
    select = Select(driver.find_element(By.ID, i[0]))
    select.select_by_visible_text(i[1])

# жмякаем на кнопку "продолжить"
continue_ = driver.find_element(By.ID, 'filters').click()
time.sleep(2)

# заполняем название группы
group = driver.find_element(By.NAME, 'fprep')
group.send_keys(speciality)

# жмякаем на кнопку "получить расписание"
get_raspis = driver.find_element(By.NAME, 'potokbut2').click()
time.sleep(2)

# записываем данные с появляющейся таблички в список data
data = []
for tr in driver.find_elements(By.XPATH, '//*[@id="dop2"]/table//tr'):
    tds = tr.find_elements(By.TAG_NAME, 'td')
    data.append([td.text for td in tds])

C:\Users\petti\AppData\Local\Temp/ipykernel_4696/2839940371.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


Введите форму обучения: Дневная
Введите курс обучения: Курс 4
Введите срок: На неделю
Введите название вашей специальности: оис


[['Дата', 'Пара', 'Группа', 'П\\гр', 'Аудит', 'Предмет', 'Тип', 'Лектор'],
 ['13.10.2022 ЧТ',
  '3',
  '-',
  '-',
  '309(2)',
  'Второй иностранный язык',
  'сем',
  'Прилепская М.В.'],
 ['704(2)', 'Второй иностранный язык', 'сем', 'Мастяница М.С.'],
 ['4',
  '-',
  '-',
  '309(2)',
  'Второй иностранный язык',
  'сем',
  'Прилепская М.В.'],
 ['428а', 'Второй иностранный язык', 'сем', 'Тамайо Кабрера'],
 ['704(2)', 'Второй иностранный язык', 'сем', 'Мастяница М.С.'],
 ['5', '-', '-', '-', 'Спец. занятость', 'спец', '-'],
 ['6', '-', '-', '-', 'Спец. занятость', 'спец', '-'],
 ['14.10.2022 ПТ',
  '1',
  '-',
  '-',
  '428а',
  'Второй иностранный язык',
  'сем',
  'Тамайо Кабрера'],
 ['2', '-', '-', '428а', 'Второй иностранный язык', 'сем', 'Тамайо Кабрера'],
 ['3', '-', '-', '-', 'Спец. занятость', 'спец', '-'],
 ['4',
  '-',
  '-',
  'ИИНТБ',
  'Элективные дисциплины по физической культуре и спорту',
  'сем',
  '-'],
 ['5', '-', '-', '-', 'Спец. занятость', 'спец', '-'],
 ['17.10.202

In [34]:
# я хочу загрузить полученные данные в pandas.DataFrame. В таблицу то есть.
import pandas as pd

# создаем датафрейм с расписанием, в качестве названий колонок берем первую строку 
data_frame = pd.DataFrame(columns=data[0])

# дальше будем добавлять в датасет данные
for i in data[1:]:
    
    # если длина списка равна 8, то он полный, поэтому дополнительно сохраним все эти строчки, потому что у некоторых строчек
    # обрезана информация, наверное, чтобы не повторяться
    # ИЛИ ПОТОМУ ЧТО РАЗРАБЫ ЕБЛАНЫ НУ КТО НАЗЫВАЕТ СУЩНОСТИ РУССКИМИ СЛОВАМИ АНГЛИЙСКИМИ БУКВАМИ БЛЯЯЯЯЯТЬ
    # тильт
    if len(i)==8:
        full_info = i
        data_frame.loc[len(data_frame.index)] = i
        
    # если список неполный, мы добавляем к нему информацию предыдущей пары
    else:
        data_frame.loc[len(data_frame.index)] = list(data_frame.loc[len(data_frame.index)-1][:8-len(i)]) + i
        
data_frame

,Дата,Пара,Группа,П\гр,Аудит,Предмет,Тип,Лектор
0,13.10.2022 ЧТ,3,-,-,309(2),Второй иностранный язык,сем,Прилепская М.В.
1,13.10.2022 ЧТ,3,-,-,704(2),Второй иностранный язык,сем,Мастяница М.С.
2,13.10.2022 ЧТ,4,-,-,309(2),Второй иностранный язык,сем,Прилепская М.В.
3,13.10.2022 ЧТ,4,-,-,428а,Второй иностранный язык,сем,Тамайо Кабрера
4,13.10.2022 ЧТ,4,-,-,704(2),Второй иностранный язык,сем,Мастяница М.С.
5,13.10.2022 ЧТ,5,-,-,-,Спец. занятость,спец,-
6,13.10.2022 ЧТ,6,-,-,-,Спец. занятость,спец,-
7,14.10.2022 ПТ,1,-,-,428а,Второй иностранный язык,сем,Тамайо Кабрера
8,14.10.2022 ПТ,2,-,-,428а,Второй иностранный язык,сем,Тамайо Кабрера
9,14.10.2022 ПТ,3,-,-,-,Спец. занятость,спец,-
